This notebook gives you an end-to-end example on how to get started using Python SDK to analyze a receipt with Azure Form Recognizer.


## Prerequistes
- Azure subscription - [Create one for free](https://azure.microsoft.com/en-us/free/cognitive-services/)
- [Python 3.x](https://www.python.org/) - Your Python installation should include [pip](https://pip.pypa.io/en/stable/). You can check if you have pip installed by running `pip --version` on the command line. Get pip by installing the latest version of Python.
- Once you have your Azure subscription, [create a Form Recognizer resource](https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesFormRecognizer) in the Azure portal to get your **key** and **endpoint**. After it deploys, click **Go to resource** - You will need the key and endpoint from the resource you create to connect your application to the Form Recognizer API. Later in the quickstart, you will paste your key and endpoint into the code below. You can use the free pricing tier (`F0`) to try the service, and upgrade later to a paid tier (`S0`) for production.

## Setting up - Install the client library
After installing Python, you can install the latest version of Form Recognier client library with:
`pip3 install --upgrade azure-ai-formrecognizer`

In [19]:
# !pip3 install --upgrade azure-ai-formrecognizer

## Get the key and endpoint
Refer to the screenshot on how to get the key and endpoint of your Form Recognizer resource.
![How to find endpoint and key](how-to-find-endpoint-and-key.png)

In [6]:
FORMRECOGNIZER_ENDPOINT = "https://demo-fr-resource.cognitiveservices.azure.com/"
FORMRECOGNIZER_KEY = "b46670bd084a4878975c77715803f7b3"

## Authenticate a FormRecognizerClient for document analysis
[FormRecognizerClient](https://docs.microsoft.com/en-us/python/api/azure-ai-formrecognizer/azure.ai.formrecognizer.formrecognizerclient?view=azure-python) is used to query the service to recongize document fields and conent like key-value pairs, tables with prebuilt or custom trained models.

In [7]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import FormRecognizerClient, FormField

# Initiate client with given endpoint and credential
client = FormRecognizerClient(FORMRECOGNIZER_ENDPOINT, AzureKeyCredential(FORMRECOGNIZER_KEY))

## Start an analyze request for your local files with `begin_recognize_receipts`

In [16]:
# Read the sample image file into memory
IMAGE_FILE = 'sample-receipt.png'
with open(IMAGE_FILE, 'rb') as f:
    data = f.read()

# Send request to Form Recognizer service to process data
task = client.begin_recognize_receipts(data)

# Get the analyze result
analyzed_result = task.result()
print('Status: {}, Document(s): {}'.format(task.status(), len(analyzed_result)))

Status: succeeded, Document(s): 1


## You can also analyze files from the web using `begin_recognize_receipts_from_url`

In [17]:
IMAGE_URL = 'https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/contoso-receipt.png'

# Send request to Form Recognizer service to process data
task = client.begin_recognize_receipts_from_url(IMAGE_URL)

# Get the analyze result
analyzed_result = task.result()
print('Status: {}, Document(s): {}'.format(task.status(), len(analyzed_result)))

Status: succeeded, Document(s): 1


## Extract information from analyzed result

In [18]:
for receipt in analyzed_result:
    for name, field in receipt.fields.items():
        if name == "Items":
            print("Receipt Items:")
            for idx, items in enumerate(field.value):
                print("...Item #{}".format(idx + 1))
                for item_name, item in items.value.items():
                    print("......{}: {} has confidence {}".format(item_name, item.value, item.confidence))
        else:
            print("{}: {} has confidence {}".format(name, field.value, field.confidence))

Receipt Items:
...Item #1
......Name: Surface Pro 6 has confidence 0.914
......Quantity: 1.0 has confidence 0.971
......TotalPrice: 999.0 has confidence 0.983
...Item #2
......Name: SurfacePen has confidence 0.718
......Quantity: 1.0 has confidence 0.976
......TotalPrice: 99.99 has confidence 0.967
MerchantAddress: 123 Main Street Redmond, WA 98052 has confidence 0.975
MerchantName: Contoso has confidence 0.974
MerchantPhoneNumber: None has confidence 0.988
ReceiptType: Itemized has confidence 0.99
Subtotal: 1098.99 has confidence 0.982
Tax: 104.4 has confidence 0.985
Total: 1203.39 has confidence 0.957
TransactionDate: 2019-06-10 has confidence 0.987
TransactionTime: 13:59:00 has confidence 0.985


## Next steps
- Learn [Receipt concept](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/concept-receipts)
- Explore the [different offerings](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/overview) in Form Recognizer
- Try Form Recognizer with [sample tool](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/quickstarts/get-started-with-form-recognizer)